In [82]:
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pathlib as pl

In [97]:
def ensure_odd(number):
    """
    Ensure the given number is odd. If the number is even, increment it by 1 to make it odd.
    
    Parameters:
    number (int): The number to check and possibly modify.
    
    Returns:
    int: An odd number.
    """
    if number % 2 == 0:
        return number + 1
    return number

In [95]:
def process(tiff_path):
    img = Image.open(tiff_path)
    img = img.copy().crop((0, 430, img.size[0], 1790))
    img = Image.fromarray(np.divide(np.array(img), 2**8 - 1)).convert("L")
    img = img.resize((img.size[0] // 30, img.size[1] // 2))
    img_np = np.array(img)
    mean = np.mean(img_np)
    std = np.std(img_np)
    print(mean, std)
    edgesR = cv2.Canny(img_np,mean-2*std,mean-1*std)
    edgesG = cv2.Canny(img_np,mean-std,mean+std)
    edgesB = cv2.Canny(img_np,mean+1*std,mean+2*std)
    # Apply Gaussian blur to the resized image

    # Detect edges in the blurred image
    
    a = (ensure_odd(int(std * 4)), ensure_odd(int(std/2)))

    edgesY = cv2.Canny(
        cv2.GaussianBlur(img_np, a, 2, 2),
        mean - 2 * std,
        mean - 1 * std,
    )
    edgesM = cv2.Canny(
        cv2.GaussianBlur(img_np, a, 2, 2), mean - 2 * std, mean + std
    )
    edgesC = cv2.Canny(
        cv2.GaussianBlur(img_np, a, 2, 2), mean - 3 * std, mean - 2 * std
    )

    # Make edgesY, edgesM, edgesC wider
    kernel = np.ones((8, 8), np.uint8)
    edgesY = cv2.dilate(edgesY, kernel, iterations=1)
    edgesM = cv2.dilate(edgesM, kernel, iterations=1)
    edgesC = cv2.dilate(edgesC, kernel, iterations=1)
    # Plot the original image and the edges

    # Create a color image from the grayscale image
    color_img = cv2.cvtColor(img_np, cv2.COLOR_GRAY2RGB)
    black_img = np.zeros_like(color_img)
    black_img[..., 0] += edgesY + edgesM
    black_img[..., 1] += edgesC + edgesY
    black_img[..., 2] += edgesM + edgesC

    old_color_img = color_img.copy()
    old_color_img = cv2.addWeighted(old_color_img, 1, black_img, 0.4, 0)

    # Add edges with specified opacity
    color_img[..., 0] = cv2.addWeighted(color_img[..., 0], 1, edgesR, 0.4, 0)
    color_img[..., 1] = cv2.addWeighted(color_img[..., 1], 1, edgesG, 0.4, 0)
    color_img[..., 2] = cv2.addWeighted(color_img[..., 2], 1, edgesB, 0.4, 0)
    fin_color_img = cv2.addWeighted(color_img, 1, black_img, 0.2, 0)

    plt.figure(figsize=(20, 10))

    plt.subplot(2, 4, 1)
    plt.title('Original Image')
    plt.imshow(img)

    plt.subplot(2, 4, 2)
    plt.title('Edges R')
    plt.imshow(edgesR)

    plt.subplot(2, 4, 3)
    plt.title('Edges G')
    plt.imshow(edgesG)

    plt.subplot(2, 4, 4)
    plt.title('Edges B')
    plt.imshow(edgesB)

    plt.subplot(2, 4, 5)
    plt.title('just black img')
    plt.imshow(old_color_img)

    plt.subplot(2, 4, 6)
    plt.title('black img')
    plt.imshow(black_img)

    plt.subplot(2, 4, 7)
    plt.title('simple edges')
    plt.imshow(color_img)

    plt.subplot(2, 4, 8)
    plt.title("Finalized Image")
    plt.imshow(fin_color_img)

    plt.tight_layout()
    plt.show()

In [ ]:
base_path = r"C:\Users\aashr\Desktop\research\glaciers\raw_data\tiffs"
tiff_files = list(pl.Path(base_path).glob("*.tiff"))
print(tiff_files)
for c,i in enumerate(tiff_files):
    print(i,c)
    process(i)